In [7]:
import os, json
import pandas as pd
import numpy as np
from PIL import Image
import cv2 as cv
import matplotlib.pyplot as plt
import math
from matplotlib import cm


In [8]:
image_list = os.listdir(r"C:\Users\adama\OneDrive\Dokument\marginalia\png")
image_path = r"C:\Users\adama\OneDrive\Dokument\marginalia\png"
df = pd.read_csv(r"C:\Users\adama\OneDrive\Dokument\marginalia\data.csv")
df.head()

,number,jsonname,photoname,width,height,class,xmin,ymin,xmax,ymax
0,189279,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3343,4157,m,633.645161,3258.064516,2585.258065,3745.161290
1,184298,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3678,4806,m,133.594595,1810.810811,2825.486486,2072.972973
2,184332,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3582,4776,m,2647.000000,194.000000,2873.315789,399.263158
3,184332,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3582,4776,m,294.368421,3351.894737,452.263158,3562.421053
4,184332,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3582,4776,m,1036.473684,3257.157895,1220.684211,3467.684211


In [9]:

def png_to_np(path):
    """
    Input: .png file
    Output: np.array
    """
    # read image
    img = cv.imread(path)
    # convert to grayscale 
    gray_scale_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    blur = cv.GuassianBlur(gray_scale_img, (5,5), 0)
    ret, threshold = cv.threshold(blur, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    return threshold

In [10]:
#reshape
def down_scale_img(img):
    new_width  = 500
    new_height = 350


    resized_image = cv.resize(img, (new_width, new_height))
    return resized_image

In [11]:
import warnings
warnings.filterwarnings('ignore')

#scale down images as well as make them black/white
new_df = pd.DataFrame()
counter = 0
for img in image_list:
    
    black_white = png_to_np(image_path + "/" + img)
    old_size = black_white.shape
    #print(old_size)
    ratio_x = (500 / old_size[1])
    ratio_y = (350 / old_size[0])
    #print(ratio_y)
    #get the coordinates of the old points
    rows = df.loc[df['number'] == int(img[:-4])]
    #new columns for reshaped coordinates
    rows["xmin_scaled"] = rows["xmin"].apply(lambda x: x*ratio_x)
    rows["xmax_scaled"] = rows["xmax"].apply(lambda x: x*ratio_x)
    rows["ymin_scaled"] = rows["ymin"].apply(lambda x: x*ratio_y)
    rows["ymax_scaled"] = rows["ymax"].apply(lambda x: x*ratio_y)
    new_df = pd.concat([new_df, rows[["number", "xmin_scaled", "xmax_scaled", "ymin_scaled", "ymax_scaled"]]])
    #print(new_df.loc[df['number'] == int(img[:-4])])
    down_scaled = down_scale_img(black_white)
    im = Image.fromarray(down_scaled)
    im.save(r"C:\Users\adama\OneDrive\Dokument\marginalia\rescaled_images" + "/" + str(img))
    counter += 1
    print(img)
    if counter % 10 == 0:
        print("Progress: " + str(round(counter/len(image_list)*100, 2)) + "%")
        




AttributeError: module 'cv2' has no attribute 'GuassianBlur'

In [ ]:
print(new_df.head())


      number  xmin_scaled  xmax_scaled  ymin_scaled  ymax_scaled
1510  183786     0.880282    31.139965    12.730834    40.395915
1511  183786   215.448944   229.753521    59.736989    70.019586
1512  183786    69.102113   438.820423   281.792110   311.415781
726   183844   143.107477   167.932243   155.801688   173.523207
442   183845   313.582647   415.592039   233.120021   277.626939


In [ ]:
#save new df to csv
new_df.to_csv(r"C:\Users\adama\OneDrive\Dokument\marginalia\rescaled_data.csv")

In [ ]:
#see if boxes are correct
image = cv.imread(r"C:\Users\adama\OneDrive\Dokument\marginalia\rescaled_images\183786.png")
window_name = 'Image'
color = (255, 0, 0)
start_point = (int(new_df.iloc[2]["xmin_scaled"]), int(new_df.iloc[2]["ymin_scaled"]))
end_point = (int(new_df.iloc[2]["xmax_scaled"]), int(new_df.iloc[2]["ymax_scaled"]))
thickness = 2

image = cv.rectangle(image, start_point, end_point, color, thickness)
cv.imshow(window_name, image) 
cv.waitKey(0)
cv.destroyAllWindows()
